<a href="https://colab.research.google.com/github/pCodePy/gColab/blob/gColab-branch-01/pyGen_DTM_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
##### First function will take datetime series
##### and split datetime object by

##### Second function can generate series 
##### with start/end periods

# -- date;
# -- year;
# -- qtr;
# -- monNo;
# -- monName;
# -- weekNo (1, 2, ..., 52);
# -- day (1, 2, ..., 7);
# -- dayMon (day number in a month: 1, 2, ..., 31);
# -- dayYr (day number in a year: 1, 2, ..., 365);
# -- dow (day of week: Mon, Tue, ..., Sun);
# -- isWeekend (1 for weekend, 0 for weekday);
# -- dayType (1 = Mondays, 2 = Tue/Wed/Thu, ...);
# -- hour
# -- isHoliday (1 for holidays, 0 for all else) => according to BG calendar;
# -- loadType (peak [CET: 9.00-20.00 / EET: 10.00-21.00], offpeak);

In [0]:
### For holidays use the below library:
# -- check: https://pypi.org/project/workalendar/
# -- and: https://pypi.org/project/holidays/
# -- which is another library for holidays

# !pip install workalendar
# from datetime import date
# from workalendar.europe import Bulgaria
# cal = Bulgaria()
# cal.holidays(2019)

In [0]:
# Get time series data to test function:

# -- mount gDrive and upload test dataset
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# Load libraries

!pip install workalendar
from workalendar.europe import Bulgaria
import numpy as np
import pandas as pd
import datetime

In [0]:
# Load test dataset
df = pd.read_excel('/content/drive/My Drive/py-mc-sim-dataset/ibexTimeSeries.xlsx', index_col=0)

In [0]:
def tsTransform(df, holCalendar=True, sumInfo=True):
    """
    df:
        takes dataframe object and checks if object is of pd.DataFrame type,
        also checks if index is of dtype: datetime;
    """
    if isinstance(df, pd.DataFrame)==True:

        if isinstance(df.index, pd.DatetimeIndex)==True:
    
            # -- create necessary dictionaries:
            strList_monNo = {1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'May',6:'Jun',
                            7:'Jul',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec'}
            strList_dayWk = {0:1, 1:2, 2:3, 3:4, 4:5, 5:6, 6:7}
            strList_dow = {0:'Mon', 1:'Tue', 2:'Wed', 3:'Thu', 4:'Fri', 5:'Sat', 6:'Sun'}
            strList_isWknd = {'Mon':0, 'Tue':0, 'Wed':0, 'Thu':0, 'Fri':0, 'Sat':1, 'Sun':1}
            strList_DayType = {'Mon':1,'Tue':2,'Wed':2,'Thu':2,'Fri':3,'Sat':4,'Sun':5}

            # -- initiate new columns:
            df['Date'] = df.index.date
            df['Year'] = df.index.to_series().apply(lambda x: x.year)
            df['Qtr'] = df.index.to_series().apply(lambda x: x.quarter)
            df['MonNo'] = df.index.to_series().apply(lambda x: x.month)
            df['MonNm'] = df['MonNo'].map(strList_monNo)
            df['WeekNo'] = df.index.week
            df['DayYr'] = df.index.dayofyear
            df['DayMn'] = df.index.to_series().apply(lambda x: x.day)
            df['DayWk'] = df.index.to_series().apply(lambda x: x.dayofweek).map(strList_dayWk)
            df['DoW'] = df.index.to_series().apply(lambda x: x.dayofweek).map(strList_dow)
            df['DayTp'] = df['DoW'].map(strList_DayType)
            df['Hour'] = df.index.to_series().apply(lambda x: x.hour)
            df['isWknd'] = df['DoW'].map(strList_isWknd)
            df['LoadTp'] = np.select(

                [df['Hour'].between(0, 8, inclusive=True), 
                df['Hour'].between(9, 20, inclusive=True)],
                ['offpeak', 'peak'],
                default='offpeak' )
            
            ### Holiday Calendar builder:

            # -- call BG holidays from workalendar:
            cal = Bulgaria()
            
            # -- get all years included in the df:
            yearList = df['Year'].unique().tolist()

            # -- initiate blank lists:
            colDate = []
            colHold = []

            # -- initiate loop:
            for x in yearList:
            
                hol = cal.holidays(x)

                for i in range(len(hol)):

                    colDate.append(pd.to_datetime(hol[i][0]))
                    colHold.append(hol[i][1])

            # -- create holiday df that will hold data:
            holidaysDF = pd.DataFrame(list(zip(colDate, colHold)), columns=['Date', 'HolName'])

            # -- get day name of the holiday
            holidaysDF['Day'] = holidaysDF['Date'].apply(lambda x: x.dayofweek)
            holidaysDF['DoW'] = holidaysDF['Day'].map(strList_dow)

            # -- create function that will convert days into boolean:
            def f(strDayCol):

                """
                function takes dataframe string column with
                days of the type: Mon/Tue/Wed...
                """
                if strDayCol == 'Mon':
                    result = 1

                elif strDayCol == 'Fri':
                    result = 1
                
                else:
                    result = 0

                return result

            # -- create list to hold conditions (1) for Easter & Christmas days:
            conditions = [
                        (holidaysDF['HolName'] == 'Easter Sunday'), (holidaysDF['HolName'] == 'Easter Monday'),
                        (holidaysDF['HolName'] == 'Christmas Eve'), (holidaysDF['HolName'] == 'Christmas Day'),
                        (holidaysDF['HolName'] == 'Christmas')
                            ]
            # -- map each of the conditions (1) with the relevant boolean:
            choices = [3, 3, 4, 4, 4]

            # -- create new col with mapped conditions vs choices:
            holidaysDF['isSpecHol'] = np.select(conditions, choices, default=0)

            # -- apply above function (f) to day of week:
            holidaysDF['isSpecHol_2'] = holidaysDF['DoW'].apply(f)

            # -- create list to hold conditions (2) to map correctly to one column:
            conditions2 = [
                        (holidaysDF['isSpecHol'] == 3), (holidaysDF['isSpecHol'] == 4),
                        (holidaysDF['isSpecHol_2'] == 1)

                            ]
            # -- map each of the conditions (2) with the relevant boolean:
            choices2 = [3, 4, 1]

            # -- create new col 'isLongHoliday' with mapped conditions vs choices:
            holidaysDF['isLngHol'] = np.select(conditions2, choices2, default=0)

            # -- drop unnecessary columns
            holidaysDF.drop(columns=['Day', 'DoW', 'isSpecHol', 'isSpecHol_2'], inplace=True)

            ### Merge and map both dataframes:

            # -- convert to datetime for the join procedure to work:
            df['Date'] = pd.to_datetime(df['Date'])
            holidaysDF['Date'] = pd.to_datetime(holidaysDF['Date'])

            # -- reset index and add it back as pandas merge drops index after join:
            mainDF = df.reset_index().merge(holidaysDF, how='left', on='Date', left_index=True).set_index('Datetime')

            # -- fill NaN values in HolName column with '0'
            mainDF['HolName'].fillna(0, inplace=True)

            # -- create isHol column with 1/0:
            mainDF['isHol'] = mainDF['HolName'].apply(lambda x: 0 if x==0 else 1)

            # -- isLngHol: remove NaN & transform to int:
            mainDF['isLngHol'].fillna(0, inplace=True)
            mainDF.isLngHol = mainDF.isLngHol.astype(int)

            if isinstance(df.index, pd.DatetimeIndex)==False:
                print('Dataframe index is not of datetime type')
        
    else:
        print('Function expects pd.DataFrame object')

    ### Generate DF based on conditional
    if sumInfo == True:

        print('---------- Dataframe columns ---------- \n')
        print(df.index)
        print('\n ---------- Dataframe columns ---------- \n')
        print(df.columns)
        print('\n ---------- Dataframe info ---------- \n')
        print(df.info())
        print('\n ---------- Inspect dataset ---------- \n')
        print(df.head())
        print('\n ---------- Any NaN values? ---------- \n')
        print(df.isnull().sum())
        print('\n ---------- Summary --------- \n')
        print(round(df.describe()))

        if holCalendar == True:
            return mainDF
        if holCalendar == False:
            return df

    else:

         if holCalendar == True:
            return mainDF
         if holCalendar == False:
            return df


In [165]:
newDF = tsTransform(df, holCalendar=True, sumInfo=False)
newDF

,Price,Date,Year,Qtr,MonNo,MonNm,WeekNo,DayYr,DayMn,DayWk,DoW,DayTp,Hour,isWknd,LoadTp,HolName,isLngHol,isHol
Datetime,,,,,,,,,,,,,,,,,,
2017-01-01 01:00:00,81.81,2017-01-01,2017,1,1,Jan,52,1,1,7,Sun,5,1,1,offpeak,New year,0,1
2017-01-01 02:00:00,44.01,2017-01-01,2017,1,1,Jan,52,1,1,7,Sun,5,2,1,offpeak,New year,0,1
2017-01-01 03:00:00,38.08,2017-01-01,2017,1,1,Jan,52,1,1,7,Sun,5,3,1,offpeak,New year,0,1
2017-01-01 04:00:00,36.75,2017-01-01,2017,1,1,Jan,52,1,1,7,Sun,5,4,1,offpeak,New year,0,1
2017-01-01 05:00:00,36.91,2017-01-01,2017,1,1,Jan,52,1,1,7,Sun,5,5,1,offpeak,New year,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-03-31 19:00:00,74.09,2019-03-31,2019,1,3,Mar,13,90,31,7,Sun,5,19,1,peak,0,0,0
2019-03-31 20:00:00,76.06,2019-03-31,2019,1,3,Mar,13,90,31,7,Sun,5,20,1,peak,0,0,0
2019-03-31 21:00:00,74.91,2019-03-31,2019,1,3,Mar,13,90,31,7,Sun,5,21,1,offpeak,0,0,0
